In [4]:
# test file change

In [2]:
import os
import pyspark

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as pysf 

sc = SparkContext('local')
spark = SparkSession(sc)

In [10]:
!ls

dataScience  titanic  titanic_prediction.ipynb	titanic.zip


In [123]:
gender_submission_file_path = "./titanic/gender_submission.csv"
train_file_path = "./titanic/train.csv"

In [170]:
genderDf = spark.read.options(header=True, inferSchema=True).csv(gender_submission_file_path)
trainDf = spark.read.options(header=True, inferSchema=True).csv(train_file_path)

In [139]:
genderDf.limit(5).show(vertical=True)

-RECORD 0----------
 PassengerId | 892 
 Survived    | 0   
-RECORD 1----------
 PassengerId | 893 
 Survived    | 1   
-RECORD 2----------
 PassengerId | 894 
 Survived    | 0   
-RECORD 3----------
 PassengerId | 895 
 Survived    | 0   
-RECORD 4----------
 PassengerId | 896 
 Survived    | 1   



In [140]:
trainDf.limit(5).show(vertical=True)

-RECORD 0---------------------------
 PassengerId | 1                    
 Survived    | 0                    
 Pclass      | 3                    
 Name        | Braund, Mr. Owen ... 
 Sex         | male                 
 Age         | 22                   
 SibSp       | 1                    
 Parch       | 0                    
 Ticket      | A/5 21171            
 Fare        | 7.25                 
 Cabin       | null                 
 Embarked    | S                    
-RECORD 1---------------------------
 PassengerId | 2                    
 Survived    | 1                    
 Pclass      | 1                    
 Name        | Cumings, Mrs. Joh... 
 Sex         | female               
 Age         | 38                   
 SibSp       | 1                    
 Parch       | 0                    
 Ticket      | PC 17599             
 Fare        | 71.2833              
 Cabin       | C85                  
 Embarked    | C                    
-RECORD 2---------------------------
 

In [176]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [142]:
trainDf.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [171]:
trainDf.select(
    [F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in trainDf.columns]
).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [172]:
trainDf = trainDf.fillna(value=-1.0, subset=["Age"])\
    .fillna(value="NOTFOUND", subset=["Embarked"])\
    .fillna(value="NOTFOUND", subset=["Cabin"])


In [173]:
trainDf.select(
    [F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in trainDf.columns]
).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [175]:
def parseCabinSection(cabin):
    return cabin[0]

In [177]:
parseCabinSectionUDF = F.udf(lambda x:parseCabinSection(x), T.StringType())   


In [178]:
trainDf = trainDf.withColumn("CabinSection", parseCabinSectionUDF(F.col("Cabin")))

In [183]:
trainDf.groupBy(F.col("CabinSection"), F.col("Survived")).count().sort(F.col("CabinSection"), F.col("Survived")).show()

+------------+--------+-----+
|CabinSection|Survived|count|
+------------+--------+-----+
|           A|       0|    8|
|           A|       1|    7|
|           B|       0|   12|
|           B|       1|   35|
|           C|       0|   24|
|           C|       1|   35|
|           D|       0|    8|
|           D|       1|   25|
|           E|       0|    8|
|           E|       1|   24|
|           F|       0|    5|
|           F|       1|    8|
|           G|       0|    2|
|           G|       1|    2|
|           N|       0|  481|
|           N|       1|  206|
|           T|       0|    1|
+------------+--------+-----+



In [180]:
trainDf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- CabinSection: string (nullable = true)



In [111]:
trainDf\
    .groupBy(F.col("Pclass"), F.col("Sex"), F.col("Survived"))\
    .count()\
    .sort(F.col("Pclass"), F.col("Sex")).show()

+------+------+--------+-----+
|Pclass|   Sex|Survived|count|
+------+------+--------+-----+
|     1|female|       1|   91|
|     1|female|       0|    3|
|     1|  male|       1|   45|
|     1|  male|       0|   77|
|     2|female|       1|   70|
|     2|female|       0|    6|
|     2|  male|       1|   17|
|     2|  male|       0|   91|
|     3|female|       1|   72|
|     3|female|       0|   72|
|     3|  male|       1|   47|
|     3|  male|       0|  300|
+------+------+--------+-----+



In [59]:
outputDf.limit(1500).show(1500)

+-----------+--------+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+

